In [1]:
import pandas as pd

# Read in data

In [2]:
filename = 'gd1_candidates.hdf5'
candidate_df = pd.read_hdf(filename, 'candidate_df')

Make HR diagram to identify coeval and same metallicity stars (assume these belonged to a globular cluster)

We have a table of Gaia data for each object that Gaia identified and a table of Pan-STARRS data. 

JOIN: match records from one table with records from another table using a key (common information)

# Preparing a table for uploading

In [4]:
from astropy.table import Table
candidate_table = Table.from_pandas(candidate_df)
type(candidate_table)

astropy.table.table.Table

In [5]:
table_id = candidate_table[['source_id']]
table_id.write('candidate_df.xml', format='votable', overwrite=True)

Note: we wrote only source_id b/c that's the only one we need, easier to transmit

In [7]:
!head candidate_df.xml

<?xml version="1.0" encoding="utf-8"?>
<!-- Produced with astropy.io.votable version 4.1
     http://www.astropy.org/ -->
<VOTABLE version="1.4" xmlns="http://www.ivoa.net/xml/VOTable/v1.4" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.4">
 <RESOURCE type="results">
  <TABLE>
   <FIELD ID="source_id" datatype="long" name="source_id"/>
   <DATA>
    <TABLEDATA>
     <TR>


Note: tells you the schema and where to find documentation --> self documenting

In [8]:
!ls -lh candidate_df.xml

-rw-r--r--  1 bostroem  staff   395K Nov 18 16:53 candidate_df.xml


**Exercise 1**: There's a gotcha here we want to warn you about. Why do you think we used double brakets to specify the column we wanted? What happens if you use single brakets?

RUN:

In [9]:
table = candidate_table[['source_id']]
type(table)

astropy.table.table.Table

In [10]:
column = candidate_table['source_id']
type(column)

astropy.table.column.Column

Double brakets gives you a table rather than a column

# Uploading a table

upload table and run query all with launch_job_async

In [12]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


simple query: download all rows and column from uploaded table tap_upload.candidate_df

In [20]:
query = """SELECT *
FROM tap_upload.candidate_df as best"""

note the use of alias `as best`

In [21]:
from astroquery.gaia import Gaia

job = Gaia.launch_job_async(query=query,
                           upload_resource='candidate_df.xml', #what we're uploading
                           upload_table_name = 'candidate_df') #what we want to call it

INFO: Query finished. [astroquery.utils.tap.core]


In [16]:
results = job.get_results()
results


source_id
int64
635559124339440000
635860218726658176
635674126383965568
635535454774983040
635497276810313600
635614168640132864
635821843194387840
635551706931167104
635518889086133376


### Check that they are the same

In [18]:
len(candidate_df), len(results)

(7346, 7346)

In [19]:
set(candidate_table['source_id']) == set(results['source_id'])

True

# Joining with an uploaded table

In [28]:
query1 = """SELECT *
FROM gaiadr2.panstarrs1_best_neighbour AS best
JOIN tap_upload.candidate_df as candidate_df
ON best.source_id = candidate_df.source_id"""

Break down each clause

In [29]:
job1 = Gaia.launch_job_async(query=query1,
                            upload_resource='candidate_df.xml',
                            upload_table_name='candidate_df')

INFO: Query finished. [astroquery.utils.tap.core]


In [40]:
results1 = job1.get_results()
results1.colnames

['source_id',
 'original_ext_source_id',
 'angular_distance',
 'number_of_neighbours',
 'number_of_mates',
 'best_neighbour_multiplicity',
 'gaia_astrometric_params',
 'source_id_2']

Has all columns in panstarrs1_best_neighbour and single column from uploaded table (source_id).
source_id occures twice --> one labeled _2 automatically

original_ext_source_id = obj_id in PAN-STARRS table

In [32]:
results1[0:5]

source_id,original_ext_source_id,angular_distance,number_of_neighbours,number_of_mates,best_neighbour_multiplicity,gaia_astrometric_params,source_id_2
,,arcsec,,,,,
int64,int64,float64,int32,int16,int16,int16,int64
635860218726658176,130911385187671349,0.053667035895467084,1,0,1,5,635860218726658176
635674126383965568,130831388428488720,0.038810268141577516,1,0,1,5,635674126383965568
635535454774983040,130631378377657369,0.034323028828991076,1,0,1,5,635535454774983040
635497276810313600,130811380445631930,0.04720255413250006,1,0,1,5,635497276810313600
635614168640132864,130571395922140135,0.020304189709964143,1,0,1,5,635614168640132864


In [33]:
len(results1)

3724

In [34]:
len(candidate_df)

7346

Not all stars in candidate_df have a match in PanSTARRS

Check that stars in Gaia catalog only have one match in PanSTARRS (check that `best_neighbour_multiplicity` is 1) with histogram

In [36]:
import pandas as pd

Count the number of times there are different multiplicities

In [38]:
nm = pd.Series(results1['best_neighbour_multiplicity'])
nm.value_counts()

1    3724
dtype: int64

# Getting the Photometry Data

**Exercise 2**
1. Make an Astropy Table that contains source_id and original_ext_source_id
2. write new table to VOTable called external.xml
3. write query to join uploaded table with gaiadr2.panstarrs1_original_valid
    * Incremental approach:
        - download all columns from upload
        - download TOP 10 rows from join
        - download full join
4. Run query
5. Click green check yes on Zoom

**Bonus**: modify query to select only the columns we need: `source_id` (from upload table), `g_mean_psf_mag` and `i_mean_psf_mag` (from `gaiadr2.panstarrs1_original_valid`)

**Challenge:** Write both joins in a single query

In [42]:
pan_id_table = results1[['source_id', 'original_ext_source_id']]
type(pan_id_table)



In [47]:
filename = 'external.xml'
pan_id_table.write(filename, format='votable', overwrite=True)

In [46]:
!head panstarrs_id.xml

<?xml version="1.0" encoding="utf-8"?>
<!-- Produced with astropy.io.votable version 4.1
     http://www.astropy.org/ -->
<VOTABLE version="1.4" xmlns="http://www.ivoa.net/xml/VOTable/v1.4" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.4">
 <RESOURCE type="results">
  <TABLE>
   <FIELD ID="source_id" datatype="long" name="source_id" ucd="meta.id;meta.main">
    <DESCRIPTION>
     Unique Gaia source identifier
    </DESCRIPTION>


In [57]:
query_phot0 = """SELECT *
FROM tap_upload.external"""

In [64]:
job_phot0 = Gaia.launch_job_async(query_phot0,
                           upload_resource='external.xml',
                          upload_table_name='external')

INFO: Query finished. [astroquery.utils.tap.core]


In [65]:
results_phot0 = job_phot0.get_results()
print(len(results_phot0), len(results1))

3724 3724


In [84]:
query_phot10 = """SELECT 
TOP 10 * 
FROM gaiadr2.panstarrs1_original_valid as ps 
JOIN tap_upload.external 
ON ps.obj_id = external.original_ext_source_id"""

In [85]:
job_phot10 = Gaia.launch_job(query_phot10, 
                          upload_resource='external.xml',
                          upload_table_name='external')

In [86]:
results_phot10 = job_phot10.get_results()
print(results_phot10)

         obj_name                obj_id       ... original_ext_source_id
                                              ...                       
-------------------------- ------------------ ... ----------------------
PSO J091404.504+190532.466 130911385187671349 ...     130911385187671349
PSO J091522.284+190154.572 130831388428488720 ...     130831388428488720
PSO J091121.063+185150.535 130631378377657369 ...     130631378377657369
PSO J091210.694+190034.188 130811380445631930 ...     130811380445631930
PSO J091822.130+184828.839 130571395922140135 ...     130571395922140135
PSO J091650.193+183726.923 130341392091279513 ...     130341392091279513
PSO J091943.991+191004.870 131001399333502136 ...     131001399333502136
PSO J091927.711+200040.863 132011398654934147 ...     132011398654934147
PSO J091654.869+184109.420 130421392285893623 ...     130421392285893623
PSO J091844.711+185252.093 130651396862917914 ...     130651396862917914


In [87]:
query_phot = """SELECT 
*
FROM gaiadr2.panstarrs1_original_valid as ps 
JOIN tap_upload.external 
ON ps.obj_id = external.original_ext_source_id"""

In [88]:
job_phot = Gaia.launch_job_async(query_phot, 
                          upload_resource='external.xml',
                          upload_table_name='external')

INFO: Query finished. [astroquery.utils.tap.core]


In [89]:
results_phot = job_phot.get_results()

In [90]:
print(len(results_phot), len(results1))

3724 3724


In [94]:
query2 = """SELECT 
external.source_id, ps.g_mean_psf_mag, ps.i_mean_psf_mag
FROM gaiadr2.panstarrs1_original_valid as ps 
JOIN tap_upload.external 
ON ps.obj_id = external.original_ext_source_id"""

In [95]:
job2 = Gaia.launch_job_async(query2, 
                          upload_resource='external.xml',
                          upload_table_name='external')

INFO: Query finished. [astroquery.utils.tap.core]


In [97]:
results2 = job2.get_results()
results2

source_id,g_mean_psf_mag,i_mean_psf_mag
,,mag
int64,float64,float64
635860218726658176,17.8978004455566,17.5174007415771
635674126383965568,19.2873001098633,17.6781005859375
635535454774983040,16.9237995147705,16.478099822998
635497276810313600,19.9242000579834,18.3339996337891
635614168640132864,16.1515998840332,14.6662998199463
635598607974369792,16.5223999023438,16.1375007629395
635737661835496576,14.5032997131348,13.9849004745483
635850945892748672,16.5174999237061,16.0450000762939


# Write the Data

astropy table --> store as fits

In [98]:
filename = 'gd1_photo.fits'
results2.write(filename, overwrite=True)

In [100]:
!ls -lh gd1_photo.fits

-rw-r--r--  1 bostroem  staff    96K Nov 18 17:59 gd1_photo.fits
